# Breaking RSA - Hastad's broadcast attack

#### Imagine a message m is encrypted differently and sent out to multiple people. 

**Hastad's broadcast attack** states as soon as the number of ciphertexts(and their corresponding public keys e, N) is approximately e, m is no longer secure. 

We can recover m using **Chinese Remainder Theorem(CRT)**.

Conditions for CRT to work:

    1. encrypted with the same small public exponent e
    2. modulus N used are relatively coprime eg. gcd(N1,N2)=1

We are showcasing this attack for:

 - **unpadded message**
 - **linearly padded message**

### Importing libraries

In [1]:
import binascii
import random
from textwrap import wrap

### Utility functions

string to ascii conversion

In [2]:
def convert_to_ascii(message):
    ascii_message = ""
    for char in message:
        ascii_char = ord(char)
        ascii_char = str(ascii_char)
        ascii_char = ascii_char.zfill(3)
        ascii_message += ascii_char
    return int(ascii_message)

ascii to string conversion

In [3]:
def convert_to_string(ascii_message):
    message = ""
    ascii_message = str(ascii_message)
    if(len(ascii_message) % 3 != 0):
        for _ in range(3 - len(ascii_message) % 3):
            ascii_message = "0" + ascii_message
    ascii_list = wrap(ascii_message, 3)
    for ascii in ascii_list:
        message = message + chr(int(ascii))
    return message

## Unpadded Message Attack

Hastad function:

In [4]:
def hastad(ciphertexts, moduli, e=3):

    if not (len(ciphertexts) == len(moduli) == e):
            raise RuntimeError("Uneven moduli and ciphertext arrays/insufficient elements !=e ")

    M = crt(ciphertexts, moduli)
    m = 10^(log(M, base=10)/e)

    return m

attack function:

In [5]:
def attack(message, key, bits):
    m = convert_to_ascii(message)
    e = key
    
    ciphertexts = []
    moduli = []
    bound = 2^bits

    for i in range(e):
        p = random_prime(bound,proof=false)
        q = random_prime(bound,proof=false)
        n = p*q
        c = Integer(pow(m, e, n))
        moduli += [Integer(n)]
        ciphertexts += [c]

    print("Moduli used")
    print(moduli)
    print("")
    print("Ciphertext outputs")
    print(ciphertexts)
    print("")
    
    m1 = convert_to_string(hastad(ciphertexts,moduli,e))
    print("Recovered message is: " + m1)
    print("Original message is: " + message)

    return

In [6]:
message = "i love sc4010"
e = 3
pqbits = 512

#debug stuffz
#moduli = [6,35,143]
#ciphertexts = [5,20,125]

attack(message, e, pqbits)

Moduli used
[32413645819554304421438637062219378016326819304326437417772383917565682116966405657700422479652509569469516120254007714727970719021518850483432014675520280020412287269708418863405251879586810777416124475672972698981933487288131554326431832916969088500165517501441045224567442055739852974855173767893272514949, 18516716207763613898345657340734766848206886086047888326362356184119379095389905678902758058268075955091128701480793112936171938516088108155145650654844066424725694641642415135762516857368177125642760025655682075780438840983142458886188456310068274879040622309385535552869825790519634895050978327973498126137, 19675686843096276698955796781232412005768179617826957695246174663685363913353112058365573177150646906066445797717061480725785735455464016243715599766409953843210729664575390047513390053580609418813501813065867672822126836798169944057745984840690251373499425755396283618653499820966589636911732041151708746081]

Ciphertext outputs
[115868730055153429758672765983878999

## Linearly Padded Message Attack

Hastad function (linear padding version):

In [7]:
def hastad_padding(ciphertexts, moduli, pad_array, const_array=(), e=3, eps=1/8):
    if not(len(ciphertexts) == len(moduli) == len(pad_array) == len(const_array) == e):
        raise RuntimeError("Uneven arrays lengths/insufficient elements !=e ")

    #initialising of arrays
    coeff_array = [Integer(0)]*e #CRT coeff
    crt_array = [Integer(0)]*e
    polynomial_array = []

    for i in range(e):
        crt_array = [0]*e
        crt_array[i] = 1
        coeff_array[i] = Integer(crt(crt_array,moduli))

    G.<x> = PolynomialRing(Zmod(prod(moduli)))
    for i in range(e):
        #polynomial f(x) = (A*x + b) ^ e - C where (A*x + b) is the padding polynomial
        polynomial_array += [coeff_array[i]*((pad_array[i]*x+const_array[i])^e - Integer(ciphertexts[i]))]

    g = sum(polynomial_array).monic()  #Creates monic polynomial, coeff of highest x deg = 1
    roots = g.small_roots(epsilon=eps)
    return roots[0] if len(roots) >= 1 else print("Message lost!")

attack function:

In [8]:
def attack2(message, key, padbits,pqbits):
    moduli = []
    ciphertexts = []
    pad_array = []
    const_array = []
    e = key
    pad_bound = 2^padbits
    prime_bound = 2^pqbits
    m = convert_to_ascii(message)

    for i in range(e):
        pad = random.randint(0,pad_bound)
        constant = random.randint(0,pad_bound)
        pad_array += [pad]
        const_array += [constant]
        p = random_prime(prime_bound,proof=false)
        q = random_prime(prime_bound,proof=false)
        n = p*q
        moduli += [n]
        ciphertexts.append(pow(pad * m + constant,e,n))
    
    print("Moduli used")
    print(moduli)
    print("")
    print("Ciphertexts outputs")
    print(ciphertexts)
    print("")
    print("padding polynomial (A*x + b)")
    print("pad array A")
    print(pad_array)
    print("")
    print("const array b")
    print(const_array)
    print("")
    
    
    m1 = convert_to_string(hastad_padding(ciphertexts, moduli, pad_array, const_array, e))
    print("Recovered message is: " + m1)
    print("Original message is: " + message)
    return

In [9]:
message = "i dislike presentation"
e = 3
padbits = 256
pqbits = 512

attack2(message, e, padbits, pqbits)

Moduli used
[53772409441914783920514901078785627755512541070269020850919599870521446597461981015929180148280818429389504634671900236511628534247985370272419047173099656988878711789777403656834031858155957770522026449381448613468553259717863843723824995255095829765437369315834565573203745071649299032940136019258126809831, 20724685941272628214108980919991494946822267898745569945108589334792001835561953619438306011991044462693032496145274937474106097508073195813741461836612742622205947484921910969736134844296684575769999792895344059746325104799921880393200534103148335832667036059212204528838392134054645010603737844520562828677, 12645210757523575567250668700262967222194743491668849825321420549849006245035045540890287904882408615107694763033958755563587387694268305970786755105674992799971187848474394151248985628648341920815998334049235490770245850782695649172588457898636533255568400476473839727893182626043103883020182015436601753699]

Ciphertexts outputs
[87954466669915884831874220514696274

## Conclusion

As seen in the 2 variations of Hastad broadcast attacks, CRT is capable of recovering the secret message m. 

These attacks can be prevented by using larger e values as there will be insufficient ciphertexts to conduct the attack.

Padding of secret message m should also be randomised instead of linearly padded(as seen by pad array A)

## References

http://koclab.cs.ucsb.edu/teaching/cren/project/2017/chennagiri.pdf

https://docs.xanhacks.xyz/crypto/rsa/08-hastad-broadcast-attack/

https://en.wikipedia.org/wiki/Coppersmith%27s_attack#:~:text=modulo%20a%20composite%20.-,H%C3%A5stad's%20broadcast%20attack,%2C%20say%20%2C%20and%20different%20moduli%20.